In [ ]:
library(circlize)
library(ComplexHeatmap)
library(ggplot2)
library(dplyr)
library(GenomicRanges)
library(stringr)

In [2]:
library(ggpubr)

### load the cytobands

In [49]:
genome = read.table('/disk1/pengweixing/database/hg38/hg38.cytoBand.clean.txt',header=F,sep="\t")
colnames(genome) = c("chr","start","end","arm","gene")

### load the phenotype

In [50]:
phe <- readRDS('../phe_113.rdata')

In [51]:
high_names <- phe[phe$group=="High",]$sample
low_names <- phe[phe$group=="Low",]$sample
neg_names <- phe[phe$group=="negative",]$sample

### load the CNA data

In [52]:
data <- read.table('All.sample.merge.cnv',header=F)

In [53]:
colnames(data) = c("chr",'start','end','cn','type','sample')

In [54]:
data$sample %>% paste0('9D',.) %>% str_replace('T','') -> data$sample

In [55]:
head(data)

,chr,start,end,cn,type,sample
,<chr>,<int>,<int>,<dbl>,<chr>,<chr>
1,chr9,35546521,36110079,3.0,gain,9DW0002
2,chr12,92929042,133234356,3.0,gain,9DW0002
3,chr17,410342,601896,5.5,gain,9DW0002
4,chr17,623532,864050,4.0,gain,9DW0002
5,chr19,29528432,39386799,3.5,gain,9DW0002
6,chr19,39388338,58571389,4.0,gain,9DW0002


### remove chrX and chrY 

In [56]:
data$chr %>% str_detect('chrY|chrX') %>% !. -> index
data2 <- data[index,]

### divide into high,Low,Neg

In [57]:
High_data <- filter(data2,sample %in%  high_names)
Low_data <- filter(data2,sample %in%  low_names)
Neg_data <- filter(data2,sample %in%  neg_names)

### visualization

In [58]:
head(High_data)

,chr,start,end,cn,type,sample
,<chr>,<int>,<int>,<dbl>,<chr>,<chr>
1,chr9,35546521,36110079,3.0,gain,9DW0002
2,chr12,92929042,133234356,3.0,gain,9DW0002
3,chr17,410342,601896,5.5,gain,9DW0002
4,chr17,623532,864050,4.0,gain,9DW0002
5,chr19,29528432,39386799,3.5,gain,9DW0002
6,chr19,39388338,58571389,4.0,gain,9DW0002


In [59]:
#select_region = data.frame(chr=c("chr9","chr15","chr19","chr21","chr13","chr14"),
 #                          start=c(38443195,85880806,20321755,10713803,96563004,44321977),
#                           end=c(78680586,99504843,29734921,19834956,113514077,60705558))
highlight_region = data.frame(chr=c("chr15","chr20","chr5"),start=c(85880806,34887374,702009),
                           end=c(99504843,66107544,39643934))
colnames(highlight_region) = c('chr','start','end')

In [60]:
lgd_points = Legend(at = c("loss", "gain"), 
    legend_gp = gpar(fill = c("#009489","#c73857")), title_position = "topleft", title = '',row_gap = unit(2, "mm"))

In [64]:
n = length(unique(High_data$sample))
all_sample = unique(High_data$sample)
pdf('High_CNA_circos.pdf',width = 6,height =6 )
circos.initializeWithIdeogram(genome)
circos.genomicTrack(High_data, ylim = c(0, n ),
                    
    panel.fun = function(region, value, ...) {
        
        value2 = value[value$type=="loss",]
        region2 = region[value$type=="loss",]
        temp_sample = unique(value2$sample)
        for(i in seq_along(temp_sample)){ 
            
            pos = match(temp_sample[i],all_sample)
            l = value2$sample == temp_sample[i]
            circos.genomicRect(region2[l,,drop=FALSE], ytop = n-pos+1+0.4, ybottom = n - pos+1-0.4 ,
            col = "#009489",border=NA)  
        }
        
        value2 = value[value$type=="gain",]
        region2 = region[value$type=="gain",]
        temp_sample = unique(value2$sample)
        
        for(i in seq_along(temp_sample)){ 
            
            pos = match(temp_sample[i],all_sample)
            l = value2$sample == temp_sample[i]
            circos.genomicRect(region2[l,,drop=FALSE], ytop = n-pos+1+0.4, ybottom = n - pos+1-0.4 ,
            col = "#c73857",border=NA)  
        }
        
},                                
,bg.border = 'black',track.height = 0.5)

#for(i in 1:nrow(highlight_region)){

#    circos.rect(highlight_region$start[i], -1, highlight_region$end[i], n+12, highlight_region$chr[i],col=add_transparency('#800080',transparency=0.9),border="#800080")

#}
#circos.genomicLink(tra[,c(1,2)], tra[,c(3,4)], col = 'red', border = NA)  
draw(lgd_points)
dev.off()
circos.clear()

png 
  2

In [65]:
n = length(unique(Neg_data$sample))
all_sample = unique(Neg_data$sample)
pdf('Neg_CNA_circos.pdf',width = 6,height = 6)
circos.initializeWithIdeogram(genome)
circos.genomicTrack(Neg_data, ylim = c(0, n ),
                    
    panel.fun = function(region, value, ...) {
        
        value2 = value[value$type=="loss",]
        region2 = region[value$type=="loss",]
        temp_sample = unique(value2$sample)
        for(i in seq_along(temp_sample)){ 
            
            pos = match(temp_sample[i],all_sample)
            l = value2$sample == temp_sample[i]
            circos.genomicRect(region2[l,,drop=FALSE], ytop = n-pos+1+0.4, ybottom = n - pos+1-0.4 ,
            col = "#009489",border=NA)  
        }
        
        value2 = value[value$type=="gain",]
        region2 = region[value$type=="gain",]
        temp_sample = unique(value2$sample)
        
        for(i in seq_along(temp_sample)){ 
            
            pos = match(temp_sample[i],all_sample)
            l = value2$sample == temp_sample[i]
            circos.genomicRect(region2[l,,drop=FALSE], ytop = n-pos+1+0.4, ybottom = n - pos+1-0.4 ,
            col = "#c73857",border=NA)  
        }
        
},                                
,bg.border = 'black',track.height = 0.5)

#for(i in 1:nrow(select_region)){
 #   print(i)
#    circos.rect(select_region$start[i], -1, select_region$end[i], n+8, select_region$chr[i],col=add_transparency('#800080',transparency=0.9),border="#800080")

#}
#circos.genomicLink(tra[,c(1,2)], tra[,c(3,4)], col = 'red', border = NA)  
draw(lgd_points)
dev.off()
circos.clear()

png 
  2

In [66]:
n = length(unique(Low_data$sample))
all_sample = unique(Low_data$sample)
pdf('Low_CNA_circos.pdf',width = 6,height = 6)
circos.initializeWithIdeogram(genome)
circos.genomicTrack(Low_data, ylim = c(0, n ),
                    
    panel.fun = function(region, value, ...) {
        
        value2 = value[value$type=="loss",]
        region2 = region[value$type=="loss",]
        temp_sample = unique(value2$sample)
        for(i in seq_along(temp_sample)){ 
            
            pos = match(temp_sample[i],all_sample)
            l = value2$sample == temp_sample[i]
            circos.genomicRect(region2[l,,drop=FALSE], ytop = n-pos+1+0.4, ybottom = n - pos+1-0.4 ,
            col = "#009489",border=NA)  
        }
        
        value2 = value[value$type=="gain",]
        region2 = region[value$type=="gain",]
        temp_sample = unique(value2$sample)
        
        for(i in seq_along(temp_sample)){ 
            
            pos = match(temp_sample[i],all_sample)
            l = value2$sample == temp_sample[i]
            circos.genomicRect(region2[l,,drop=FALSE], ytop = n-pos+1+0.4, ybottom = n - pos+1-0.4 ,
            col = "#c73857",border=NA)  
        }
        
},                                
,bg.border = 'black',track.height = 0.5)

#for(i in 1:nrow(select_region)){
 #   print(i)
#    circos.rect(select_region$start[i], -1, select_region$end[i], n+12, select_region$chr[i],col=add_transparency('#800080',transparency=0.9),border="#800080")

#}
#circos.genomicLink(tra[,c(1,2)], tra[,c(3,4)], col = 'red', border = NA)  
draw(lgd_points)
dev.off()
circos.clear()

png 
  2